# Интро и полезная литература

### Литература

- [Word2vec в картинках](https://habr.com/ru/post/446530/). Статья на хабре от 2019 года.
- [Презентация](http://www.machinelearning.ru/wiki/images/b/b3/Word2Vec.pdf) (2017 год) с примерами реализации кода на библиотеке gensim
 
Так как Word2Vec - это уже не просто единичная реализация, а группа алгоритмов для получения векторных представлений слов, то существует масса различных пакетов на эту тему. Мы будем пользоваться gensim.

- [Gensim – Руководство для начинающих](https://webdevblog.ru/gensim-rukovodstvo-dlya-nachinajushhih/). Здесь нужно изучить ряд терминов, чтобы понимать принцип работы библиотеки. Статья 2019 года.
- [A Beginner’s Guide to Word Embedding with Gensim Word2Vec Model](https://towardsdatascience.com/a-beginners-guide-to-word-embedding-with-gensim-word2vec-model-5970fa56cc92). Статья 2019 года. По ней хорошо учиться писать код.

### Термины

- **Токен** обычно означает «слово»
- **Документ** обычно может относиться как к «предложению» так и к «абзацу»
- **Корпус** обычно представляет собой «собрание документов» в виде пакета слов или как его еще называют мешка слов 

# Импорт локальных констант и функций

Вначале перейдём на уровень рутового пути репозитория
(в дальнейшем всегда пишем пути от корня репозитория)

In [ ]:
cd ..

Заимпортируем всё, что нам пригодится в дальнейшем

In [ ]:
from shared.helpers import (
    filter_tokens_fo_Word2Vec,
    get_tokenized_scripts_json_paths, # получаем список путей к json файлам с токенами
    transform_json_file_to_train_data  # преобразует к формату, который принимает word2Vec модель
)

from shared.consts import PATH_TO_WORD2VEC_MODEL

Также внешние библиотеки подключим сразу

In [ ]:
from os import walk
import pandas as pd
import json

from gensim.models.word2vec import Word2Vec
from multiprocessing import cpu_count
from tqdm.notebook import tqdm

# Готовим данные перед построением модели

Так как токенизация была разбита на микрозадачи (чтобы хватило мощностей компьютера), то и json у нас много теперь.

In [ ]:
tokenized_scripts_jsons_paths = get_tokenized_scripts_json_paths()

Можно было бы пытаться объединить их все в одну, но опять же тут в зависимости от мощностей компьютера, может просто не хватить оперативки держать столько данных в памяти.

Благо word2Vec может учить постепенно: вначале на одних данных, потом "доучить" другим набором данных.

Поэтому вначале учим на первом наборе данных, а потом доучиваем оставшимися постепенно.

**ВАЖНОЕ УТОЧНЕНИЕ:**
Формируется словарь только на первом этапе обучения. Дальше он будет фиксированнным и будут лишь уточняться веса при online learning / resume learning.

In [ ]:
NUMBER_JSON_FILES_TO_FORM_VOCABULARY = 3
tokenized_scripts_list_of_lists = []

for json_index in tqdm(range(NUMBER_JSON_FILES_TO_FORM_VOCABULARY)):
    train_json_path = tokenized_scripts_jsons_paths[json_index]

    tokenized_scripts_list_of_lists = [
        *tokenized_scripts_list_of_lists,
        *transform_json_file_to_train_data(train_json_path)
    ]

In [ ]:
len(tokenized_scripts_list_of_lists)

# Build Word2Vec model

Теперь бы нужно построить модель и настроить гиперпараметры.

[Эта статья](https://habr.com/ru/post/446530/) говорит, что embedding_size устанавливают обычно на 300.

In [ ]:
word2VecModel = Word2Vec(
    # list of lists. Например: [["cat", "say", "meow"], ["dog", "say", "woof"]]
    sentences=tokenized_scripts_list_of_lists,
    
    # The number of dimensions of the embeddings and the default is 100
    size=300, 
    
    #The maximum distance between a target word and words around the target word. The default window is 5.
    window=10,
    
    # финальный размер словаря
    max_final_vocab=50000, 
    
    # игнорировать слова с частотностью ниже, чем эта (регулируется автоматически, если установлено max_final_vocab)
    min_count = 5, 
    
    workers=cpu_count(),
    
    # The training algorithm, either CBOW(0) or skip gram(1). The default training algorithm is CBOW
    sg=0
)

Смотрим, какие наиболее близкие вектора будут к функции с названием **setTimeout** (это встроенная функция в js, которая на вход принимает функцию первым аргументом, которую нужно исполнить через заданное кол-во времени в миллисекундах, которые будут переданы вторым аргументом).

Число напротив каждого слова - процент близости векторов.

In [ ]:
word2VecModel.wv.most_similar('setTimeout', topn=20)

Видим, что модель на маленьком кусочке данных ($\frac{1}{43}$ от всех токенов) показывает крайне странные результаты. Да, в каждом слове можно "натянуть" смысл, но это все будет не то.

Посмотрим лучше, какие будут результаты у модели, когда она "доучиться" на остальных данных.

# Resume learning

API библиотеки gensim предоставил возможность продолжать обучение модели на новых данных

In [ ]:
additional_data_jsons_paths = tokenized_scripts_jsons_paths[NUMBER_JSON_FILES_TO_FORM_VOCABULARY:]

for jsonPathIndex in tqdm(range(len(additional_data_jsons))):
    try:
        new_data = transform_json_file_to_train_data(additional_data_jsons_paths[jsonPathIndex])
    except:
        print('Не получилось расспарсить json:', additional_data_jsons_paths[jsonPathIndex])
        continue
    
    word2VecModel.build_vocab(new_data, update=True)
    word2VecModel.train(new_data, total_examples=word2VecModel.corpus_count, epochs=word2VecModel.epochs)

In [ ]:
word2VecModel.wv.most_similar('setTimeout', topn=20)

Результаты стали гораздо более впечатляющими!

- clearTimeout - обрывает запланированный вызов функции
- timeout - через какое время исполнить функцию (один из агрументов всегда)

# Сохранение и загрузка модели

In [ ]:
word2VecModel.save(PATH_TO_WORD2VEC_MODEL)

In [ ]:
from gensim.models.word2vec import Word2Vec
from shared.consts import PATH_TO_WORD2VEC_MODEL

word2VecModel = Word2Vec.load(PATH_TO_WORD2VEC_MODEL)

In [ ]:
word2VecModel.wv.most_similar('setTimeout', topn=20)

In [ ]:
print('В нашем словаре теперь', len(word2VecModel.wv.vocab), 'слов')